In [1]:
import pandas as pd
import re
import yaml
import json
import os
import configparser
from deepdiff import DeepDiff

In [6]:
def excel_to_conf(excel_file, conf_file):
        # Đọc dữ liệu từ file Excel (giả sử có 2 cột: key và value)
        df = pd.read_excel(excel_file)

        # Đảm bảo rằng thư mục 'new_json' tồn tại
        os.makedirs("folder_name_NEW", exist_ok=True)

        # Đường dẫn đến file mới trong thư mục 'new_json'
        conf_file = os.path.join("folder_name_NEW", conf_file)

        # Mở file .conf để ghi dữ liệu
        with open(conf_file, 'w') as conf:
            conf.write(f"{'[Service]'}\n")
            # Duyệt qua từng dòng của DataFrame
            for index, row in df.iterrows():
                key = row['Parameter']   # Cột key trong file Excel
                value = row['Setup Value']  # Cột value trong file Excel
                if pd.isna(value):  # Kiểm tra nếu value là NaN (trống)  
                    value = ""  # Gán None vào value thay vì bỏ qua 
                conf.write(f"{key}={value}\n")

In [7]:
excel_file = '/workspaces/test/10cu/10-kubeadm.conf.xlsx'
conf_file = 'main.conf'
excel_to_conf(excel_file, conf_file)

In [10]:
def convert_excel_to_yaml(excel_file, yaml_file, folder_name_NEW):
    df = pd.read_excel(excel_file)

    yaml_dict = {}
    
    for _, row in df.iterrows():
        keys = row["Parameter"].split(".")
        value = row["Setup Value"]

        flag = False

        # # Chuyển đổi giá trị thành kiểu phù hợp  
        # if pd.isna(value):  # Kiểm tra nếu value là NaN (trống)  
        #     value = None  # Gán None vào value thay vì bỏ qua  
        # elif isinstance(value, str) and value.upper() == "TRUE":  
        #     value = True  
        # elif isinstance(value, str) and value.upper() == "FALSE":  
        #     value = False  
        # elif isinstance(value, float) and value.is_integer():  
        #     value = int(value) 
        # elif value == "0"

        if pd.isna(value):  # Kiểm tra nếu value là NaN (trống)  
            value = None  # Gán None vào value thay vì bỏ qua  
        elif isinstance(value, str) and value.upper() == "TRUE":  
            value = True  
        elif isinstance(value, str) and value.upper() == "FALSE":  
            value = False  
        elif value == {}:  # Kiểm tra nếu value là từ điển rỗng
            pass  # Giữ nguyên, không thay đổi
        elif value == "":  # Kiểm tra nếu value là chuỗi rỗng
            pass  # Giữ nguyên, không thay đổi
        elif isinstance(value, float) and value.is_integer():  
            value = int(value)
        


        temp = yaml_dict
       
        for key in keys[:-1]:
            if key == "kubeadm":
                index = keys.index(key)  
                spec = '.'.join(keys[index:])
                # last_key = row["Parameter"]  
                temp[spec] = value
                flag = True
                break 

            if "[" in key and "]" in key: 
                base_key, index = key.split("[")
                index = int(index.rstrip("]"))
                temp = temp.setdefault(base_key, [])
                while len(temp) <= index:
                    temp.append({})
                temp = temp[index]
            else:
                temp = temp.setdefault(key, {})

        if flag:
            continue

        last_key = keys[-1]
        if "[" in last_key and "]" in last_key: 
            base_key, index = last_key.split("[")
            index = int(index.rstrip("]"))
            temp = temp.setdefault(base_key, [])
            while len(temp) <= index:
                temp.append(None)
            temp[index] = value
        else:
            temp[last_key] = value

    # Đảm bảo rằng thư mục 'new_json' tồn tại
    os.makedirs(folder_name_NEW, exist_ok=True)

    # Đường dẫn đến file mới trong thư mục 'new_json'
    yaml_file = os.path.join(folder_name_NEW, yaml_file)

    with open(yaml_file, "w") as f:
        yaml.dump(yaml_dict, f, default_flow_style=False, sort_keys=False)

    print(f"Done!")

In [11]:
input_excel = "/workspaces/test/ser.xlsx" 
output_yaml = "main1.yaml"  
folder_name_NEW ="oooooooo"
convert_excel_to_yaml(input_excel, output_yaml, folder_name_NEW)

Done!


In [ ]:

def convert_yaml_to_service(yaml_file, service_file):
    with open(yaml_file, 'r') as f:
        yaml_data = yaml.safe_load(f)
    
    with open(service_file, 'w') as f:
        # Write [Unit] section
        f.write("[Unit]\n")
        for key, value in yaml_data.get('Unit', {}).items():
            f.write(f"{key}={value}\n")
        
        # Write [Service] section
        f.write("\n[Service]\n")
        for key, value in yaml_data.get('Service', {}).items():
            f.write(f"{key}={value}\n")

        f.write("\n[Install]\n")
        for key, value in yaml_data.get('Install', {}).items():
            f.write(f"{key}={value}\n")

# Usage example
convert_yaml_to_service('main1.yaml', 'main1.service')


In [47]:
import json
import re
from deepdiff import DeepDiff

def clean_conf_file(file_path):
    """Đọc file và loại bỏ các dòng comment và khoảng trắng."""
    with open(file_path, 'r') as f:
        lines = f.readlines()

    cleaned_lines = []
    for line in lines:
        # Bỏ qua các dòng comment (bắt đầu bằng #) và các dòng trống
        if line.strip() and not line.strip().startswith('#'):
            cleaned_lines.append(line.strip())

    return "\n".join(cleaned_lines)

def compare_conf_files(file1, file2):
    result = {}

    # Làm sạch các file bằng cách loại bỏ comment và dòng trống
    conf1 = clean_conf_file(file1)
    conf2 = clean_conf_file(file2)

    # So sánh hai file đã làm sạch
    diff = DeepDiff(conf1, conf2)

    # Nếu không có sự khác biệt
    if not diff:
        result["message"] = "Same"
    else:
        result["message"] = "Different"
        
        # Nếu có sự thay đổi giá trị
        if 'values_changed' in diff:
            result["values_changed"] = []
            for key, change in diff['values_changed'].items():
                result["values_changed"].append({
                    "key": key,
                    "old_value": change['old_value'],
                    "new_value": change['new_value']
                })

        # Nếu có mục được thêm vào
        if 'dictionary_item_added' in diff:
            result["dictionary_item_added"] = []
            for key in diff['dictionary_item_added']:
                result["dictionary_item_added"].append({
                    "key": key,
                })

        # Nếu có mục bị xóa
        if 'dictionary_item_removed' in diff:
            result["dictionary_item_removed"] = []
            for key in diff['dictionary_item_removed']:
                result["dictionary_item_removed"].append({
                    "key": key,
                })

    return result

# Đường dẫn đến các file .conf
file1 = '/workspaces/test/10cu/10-kubeadm.conf'
file2 = '/workspaces/test/main.conf'

# So sánh và in kết quả
result = compare_conf_files(file1, file2)
print(result)

with open("compare.json", 'w') as json_file: 
        json.dump(result, json_file, indent=4)

{'message': 'Different', 'values_changed': [{'key': 'root', 'old_value': '[Service]\nEnvironment="KUBELET_KUBECONFIG_ARGS=--bootstrap-kubeconfig=/etc/kubernetes/bootstrap-kubelet.conf --kubeconfig=/etc/kubernetes/kubelet.conf"\nEnvironment="KUBELET_CONFIG_ARGS=--config=/var/lib/kubelet/config.yaml"\nEnvironmentFile=-/var/lib/kubelet/kubeadm-flags.env\nEnvironmentFile=-/etc/default/kubelet\nExecStart= 9999\nExecStart=/usr/bin/kubelet $KUBELET_KUBECONFIG_ARGS $KUBELET_CONFIG_ARGS $KUBELET_KUBEADM_ARGS $KUBELET_EXTRA_ARGS', 'new_value': '[Service]\nEnvironment="KUBELET_KUBECONFIG_ARGS=--bootstrap-kubeconfig=/etc/kubernetes/bootstrap-kubelet.conf --kubeconfig=/etc/kubernetes/kubelet.conf"\nEnvironment="KUBELET_CONFIG_ARGS=--config=/var/lib/kubelet/config.yaml"\nEnvironmentFile=-/var/lib/kubelet/kubeadm-flags.env\nEnvironmentFile=-/etc/default/kubelet\nExecStart=\nExecStart=/usr/bin/kubelet $KUBELET_KUBECONFIG_ARGS $KUBELET_CONFIG_ARGS $KUBELET_KUBEADM_ARGS $KUBELET_EXTRA_ARGS'}]}


In [ ]:
import json
from deepdiff import DeepDiff

def clean_service_file(file_path):
    """Đọc file .service và loại bỏ các dòng comment và khoảng trắng."""
    with open(file_path, 'r') as f:
        lines = f.readlines()

    cleaned_lines = []
    for line in lines:
        # Bỏ qua các dòng comment (bắt đầu bằng #) và các dòng trống
        if line.strip() and not line.strip().startswith('#'):
            cleaned_lines.append(line.strip())

    return "\n".join(cleaned_lines)

def compare_service_files(file1, file2):
    result = {}

    # Làm sạch các file bằng cách loại bỏ comment và dòng trống
    service1 = clean_service_file(file1)
    service2 = clean_service_file(file2)

    # So sánh hai file đã làm sạch
    diff = DeepDiff(service1, service2)

    # Nếu không có sự khác biệt
    if not diff:
        result["message"] = "Same"
    else:
        result["message"] = "Different"
        
        # Nếu có sự thay đổi giá trị
        if 'values_changed' in diff:
            result["values_changed"] = []
            for key, change in diff['values_changed'].items():
                result["values_changed"].append({
                    "key": key,
                    "old_value": change['old_value'],
                    "new_value": change['new_value']
                })

        # Nếu có mục được thêm vào
        if 'dictionary_item_added' in diff:
            result["dictionary_item_added"] = []
            for key in diff['dictionary_item_added']:
                result["dictionary_item_added"].append({
                    "key": key,
                })

        # Nếu có mục bị xóa
        if 'dictionary_item_removed' in diff:
            result["dictionary_item_removed"] = []
            for key in diff['dictionary_item_removed']:
                result["dictionary_item_removed"].append({
                    "key": key,
                })

    return result  # Trả về đối tượng Python, không phải chuỗi JSON

# Đường dẫn đến các file .service
file1 = '/workspaces/test/containerd.service'
file2 = '/workspaces/test/main1.service'

# So sánh và lưu kết quả vào file JSON
result1 = compare_service_files(file1, file2)

# Ghi kết quả vào file .json
with open("compare11111.json", 'w') as json_file:
    json.dump(result1, json_file, indent=4)

print("Kết quả đã được lưu vào file compare11111.json")

/workspaces/k8sconfig/test/new_file_default

Kết quả đã được lưu vào file compare11111.json
